In [0]:
import tensorflow as tf
from numpy.random import RandomState

#### 1. 定义神经网络的参数，输入和输出节点。

In [0]:
batch_size = 8
w1= tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2= tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))
x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")
y_= tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

#### 2. 定义前向传播过程，损失函数及反向传播算法。

In [0]:
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0))
                                + (1 - y_) * tf.log(tf.clip_by_value(1 - y, 1e-10, 1.0)))
"""
ignature: tf.clip_by_value(t, clip_value_min, clip_value_max, name=None)
Docstring:
Clips tensor values to a specified min and max.

Given a tensor `t`, this operation returns a tensor of the same type and
shape as `t` with its values clipped to `clip_value_min` and `clip_value_max`.
Any values less than `clip_value_min` are set to `clip_value_min`. Any values
greater than `clip_value_max` are set to `clip_value_max`.

Args:
  t: A `Tensor`.
  clip_value_min: A 0-D (scalar) `Tensor`, or a `Tensor` with the same shape
    as `t`. The minimum value to clip by.
  clip_value_max: A 0-D (scalar) `Tensor`, or a `Tensor` with the same shape
    as `t`. The maximum value to clip by.
  name: A name for the operation (optional).

Returns:
  A clipped `Tensor`.

"""
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

####  3. 生成模拟数据集。

In [0]:
rdm = RandomState(1)
X = rdm.rand(128,2)
Y = [[int(x1+x2 < 1)] for (x1, x2) in X]

#### 4. 创建一个会话来运行TensorFlow程序。

In [5]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    # 输出目前（未经训练）的参数取值。
    print(sess.run(w1))
    print(sess.run(w2))
    print("\n")
    
    # 训练模型。
    STEPS = 5000
    for i in range(STEPS):
        start = (i*batch_size) % 128
        end = (i*batch_size) % 128 + batch_size
        sess.run([train_step, y, y_], feed_dict={x: X[start:end], y_: Y[start:end]})
        if i % 1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x: X, y_: Y})
            print("After %d training step(s), cross entropy on all data is %g" % (i, total_cross_entropy))
    
    # 输出训练后的参数取值。
    print("\n")
    print(sess.run(w1))
    print(sess.run(w2))

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]


After 0 training step(s), cross entropy on all data is 1.89805
After 1000 training step(s), cross entropy on all data is 0.655075
After 2000 training step(s), cross entropy on all data is 0.626172
After 3000 training step(s), cross entropy on all data is 0.615096
After 4000 training step(s), cross entropy on all data is 0.610309


[[ 0.0247699   0.5694868   1.6921941 ]
 [-2.197735   -0.23668918  1.1143898 ]]
[[-0.4554471 ]
 [ 0.49110925]
 [-0.9811034 ]]
